# Lezione 2: Agenti Avanzati e Integrazioni

Benvenuti alla seconda lezione! In questo notebook imparerete:

1. 📝 **Summarize Middleware**: Riassumere conversazioni lunghe automaticamente
2. 🤝 **Human-in-the-Loop**: Intervento umano nelle decisioni dell'agente
3. 🗄️ **SQL Agent**: Interrogare database con linguaggio naturale
4. 📚 **RAG**: Retrieval Augmented Generation con PDF e Faiss

## Setup Iniziale

Verifichiamo l'ambiente e carichiamo le dipendenze.

In [ ]:
import sys
import os
from dotenv import load_dotenv

# Carica variabili d'ambiente
load_dotenv()

print(f"Python version: {sys.version}")

# Verifica API keys
if os.getenv("OPENAI_API_KEY"):
    print("✅ OpenAI API key trovata")
else:
    print("⚠️ OpenAI API key non trovata - necessaria per utilizzare i modelli OpenAI")
    print("   Ottieni una chiave su https://platform.openai.com/account/api-keys")

In [ ]:
# Inizializza il modello
from langchain.chat_models import init_chat_model

model = init_chat_model(
    "openai:gpt-4o-mini",  # Usiamo un modello più potente per questi task
    temperature=0
)

print("✅ Modello inizializzato")

In [ ]:
# con Cerebras
#from langchain_cerebras import ChatCerebras

#model = ChatCerebras(model_name="gpt-oss-120b", temperature=0)

## 1. 📝 Summarize Middleware

Per conversazioni lunghe, la **SummarizationMiddleware** riassume automaticamente i messaggi più vecchi per risparmiare token e mantenere il contesto gestibile.

**Quando usarlo:**
- Conversazioni molto lunghe (> 20-30 messaggi)
- Limiti di contesto del modello
- Costi elevati per token

- Token counter configurabile

**Novità con la nuova API:**- Gestione automatica di coppie AI/Tool message

- Middleware ufficiale da `langchain.agents.middleware.summarization`- Supporto per trigger multipli (token, messaggi, frazione)

In [ ]:
# Esempio di configurazione avanzata con trigger multipli
from langchain.agents.middleware.summarization import SummarizationMiddleware

# Trigger quando SI VERIFICA UNA delle condizioni:
advanced_summarization = SummarizationMiddleware(
    model=model,
    trigger=[
        ("messages", 50),   # O quando raggiungi 50 messaggi
        ("tokens", 4000),   # O quando raggiungi 4000 token
        #("fraction", 0.8)   # O quando usi l'80% del contesto del modello se supportato
    ],
    keep=("tokens", 2000),  # Mantieni gli ultimi 2000 token
    # token_counter: funzione custom per contare i token (opzionale)
)

print("✅ SummarizationMiddleware avanzato creato")
print("\n🔮 Modalità di trigger disponibili:")
print("   - ('messages', N): Numero di messaggi")
print("   - ('tokens', N): Numero di token assoluti")
print("   - ('fraction', F): Frazione del contesto del modello (0.0-1.0)")
print("\n🔮 Modalità di keep disponibili:")
print("   - ('messages', N): Mantieni N messaggi recenti")
print("   - ('tokens', N): Mantieni N token recenti")
print("   - ('fraction', F): Mantieni F frazione del contesto")

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langchain.agents import create_agent
from langchain.agents.middleware.summarization import SummarizationMiddleware

# Creiamo un checkpointer per la memoria
memory = MemorySaver()

# Creiamo il middleware di summarization
summarization_middleware = SummarizationMiddleware(
    model=model,
    # Trigger: quando la conversazione raggiunge 5 messaggi
    trigger=("messages", 5),
    # Keep: mantieni gli ultimi 2 messaggi dopo il riassunto
    keep=("messages", 2)
)


print("✅ Middleware di summarization creato")
print(f"   - Keep: {summarization_middleware.keep}")
print(f"   - Trigger: {summarization_middleware.trigger}")

In [ ]:
# Creiamo un agente con summarization middleware
agent_with_summary = create_agent(
    model=model,
    tools=[],
    middleware=[summarization_middleware],
    checkpointer=memory,
    system_prompt="""Sei un assistente che mantiene conversazioni lunghe.
    
Grazie al middleware di summarization, posso gestire conversazioni
di centinaia di messaggi senza perdere il contesto o superare i
limiti di token del modello.

Rispondi sempre in italiano."""
)

print("✅ Agente con SummarizationMiddleware creato!")
print("\n📚 Comportamento:")
print("   1. Quando la conversazione raggiunge 5 messaggi")
print("   2. Il middleware crea automaticamente un riassunto dei primi 30")
print("   3. Mantiene solo gli ultimi 2 messaggi + il riassunto")
print("   4. Riduce i token e mantiene il contesto rilevante")

In [ ]:
from langchain.messages import HumanMessage, AIMessage
from uuid import uuid4

config = {"configurable": {"thread_id": str(uuid4())}}

response = agent_with_summary.invoke(
    {"messages": [
        HumanMessage(content="Ciao!"),
        AIMessage(content="Ciao! Come posso aiutarti oggi?"),
        HumanMessage(content="Sai qual è il senso della vita?"),
        AIMessage(content="Certo! Il senso della vita è 42, ovviamente."),
        HumanMessage(content="Cioè? Puoi spiegarti meglio?"),
        AIMessage(content="Beh, è una risposta filosofica tratta da 'Guida Galattica per Autostoppist'. Però io ci credo davvero."),
        HumanMessage(content="Interessante! Non pensavo che le intelligenze artificiali potessero avere opinioni filosofiche."),
    ]}, 
    config=config)

print("🤖 Risposta dell'agente con summarization:\n")
print(response["messages"][-1].content)

In [ ]:
for r in response['messages']:
    r.pretty_print()

## 2. 🤝 Human-in-the-Loop Middleware

**Human-in-the-loop** permette all'agente di chiedere conferma prima di eseguire azioni sensibili.

**Casi d'uso:**
- Operazioni critiche (cancellazioni, pagamenti)
- Decisioni ambigue
- Approvazioni workflow

In [ ]:
from langchain_core.tools import tool

@tool
def delete_file(filename: str) -> str:
    """Elimina un file dal sistema. ATTENZIONE: operazione irreversibile!
    
    Args:
        filename: Nome del file da eliminare
    """
    # In un sistema reale, qui ci sarebbe la logica di eliminazione
    return f"⚠️ SIMULAZIONE: File '{filename}' sarebbe stato eliminato"

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Invia una email.
    
    Args:
        to: Destinatario
        subject: Oggetto
        body: Corpo del messaggio
    """
    return f"📧 SIMULAZIONE: Email inviata a {to}\nOggetto: {subject}"

print("✅ Tools sensibili creati")

In [ ]:
# Implementazione Human-in-the-Loop con interrupt() di LangGraph
from langgraph.types import interrupt

@tool
def delete_file_with_approval(filename: str) -> str:
    """Elimina un file dal sistema con approvazione umana richiesta.
    
    Args:
        filename: Nome del file da eliminare
    """
    # Richiedi approvazione usando interrupt()
    approval = interrupt(
        {
            "action": "delete_file",
            "filename": filename,
            "message": f"⚠️ Vuoi davvero eliminare '{filename}'? Questa operazione è irreversibile."
        }
    )
    
    if approval and approval.get("approved"):
        return f"✅ File '{filename}' eliminato con successo"
    else:
        return f"❌ Eliminazione di '{filename}' annullata"



@tool
def send_email_with_approval(to: str, subject: str, body: str) -> str:
    """Invia una email con approvazione umana richiesta.

    Args:

        to: Destinatario

        subject: Oggetto        

        body: Corpo del messaggio    

    """       
    approval = interrupt({
        "action": "send_email",
        "to": to,
        "message": f"📧 Vuoi inviare questa email a {to}?",
        "subject": subject,
        "body": body,
    })
        # Richiedi approvazione usando interrupt()  
    if approval and approval.get("approved"):
        return f"📧 Email inviata con successo a {to}"
    else:
        return f"❌ Invio email a {to} annullato"

print("- L'esecuzione dell'agente viene sospesa")
print("\n💡 Con interrupt() di LangGraph:")
print("- L'applicazione può chiedere conferma all'utente")
print("✅ Tools con Human-in-the-Loop creati") 

In [ ]:
# Creiamo un agente con Human-in-the-Loop
from langgraph.types import Command


hil_agent = create_agent(
    model=model,
    tools=[delete_file_with_approval, send_email_with_approval],
    checkpointer=memory,  # Necessario per gestire gli interrupt
    system_prompt="""Sei un assistente che esegue operazioni sensibili.

Quando l'utente ti chiede di eliminare un file o inviare un'email:
1. USA IMMEDIATAMENTE il tool appropriato (delete_file_with_approval o send_email_with_approval)
2. Il tool stesso gestirà la richiesta di approvazione con interrupt()
3. NON chiedere conferma con un messaggio - usa direttamente il tool

Rispondi sempre in italiano."""

)
print("   1. Esegui: ")
config = {'configurable': {'thread_id': str(uuid4())}}
print("   2. Prima invocazione: ")
response = hil_agent.invoke({'messages': 'Elimina report_vecchio.pdf'}, config)
print("   3. L'agente si ferma e restituisce un interrupt")
for r in response['messages']:
    r.pretty_print()

In [ ]:
print("   4. Riprendi con: ")
response = hil_agent.invoke(Command(resume={'approved': True}), config)

print(f"\n📊 Numero totale di messaggi: {len(response['messages'])}")
print(f"Tipi di messaggi: {[msg.type for msg in response['messages']]}\n")

for r in response['messages']:
    r.pretty_print()

## 3. 🗄️ SQL Agent

Un **SQL Agent** può interrogare database usando linguaggio naturale.

**Database Chinook**: Database di esempio che simula un negozio di musica digitale con:
- Artisti, Album, Brani
- Clienti, Fatture, Ordini
- Dipendenti

In [ ]:
# Verifica che il database esista
import os

db_path = "../data/resources/Chinook.db"
if os.path.exists(db_path):
    print(f"✅ Database trovato: {db_path}")
    print(f"   Dimensione: {os.path.getsize(db_path) / 1024:.1f} KB")
else:
    print(f"❌ Database non trovato: {db_path}")

In [ ]:
# Connettiamoci al database
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

print("✅ Connesso al database\n")
print("📊 Tabelle disponibili:")
print(db.get_usable_table_names())

In [ ]:
# Esploriamo la struttura del database
print("🔍 Schema della tabella 'Artist':\n")
print(db.get_table_info(["Artist"]))

print("\n🔍 Schema della tabella 'Album':\n")
print(db.get_table_info(["Album"]))

In [ ]:
# Test query SQL diretta
result = db.run("SELECT * FROM Artist LIMIT 5")
print("📝 Primi 5 artisti:\n")
print(result)

In [ ]:
# Creiamo tools per SQL
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=model)
sql_tools = toolkit.get_tools()

print(f"✅ {len(sql_tools)} SQL tools creati:\n")
for tool in sql_tools:
    print(f"  - {tool.name}: {tool.description[:80]}...")

In [ ]:
# Creiamo un SQL Agent con la nuova API
sql_agent = create_agent(
    model=model,
    tools=sql_tools,
    system_prompt="""Sei un esperto analista di database.

Quando l'utente fa una domanda sui dati:
1. Esamina lo schema delle tabelle rilevanti
2. Costruisci la query SQL appropriata
3. Esegui la query
4. Interpreta i risultati in modo chiaro

⚠️ IMPORTANTE:
- Usa LIMIT per query esplorative
- Controlla sempre i risultati prima di fare operazioni DML
- Se non sei sicuro, chiedi conferma

Rispondi sempre in italiano."""
)

print("✅ SQL Agent creato con la nuova API!")

In [ ]:
# Test SQL Agent
response = sql_agent.invoke({
    "messages": "Quali sono i 5 artisti con più album nel database?"
})

print("🤖 Risposta SQL Agent:\n")
print(response["messages"][-1].content)

In [ ]:
# Altra query di esempio
response = sql_agent.invoke({
    "messages": "Qual è il totale delle vendite per paese?"
})

print("🤖 Risposta SQL Agent:\n")
print(response["messages"][-1].content)

## 4. 📚 RAG: Retrieval Augmented Generation

**RAG** combina:
1. **Retrieval**: Cerca documenti rilevanti da una knowledge base
2. **Augmented**: Arricchisce il prompt con informazioni recuperate
3. **Generation**: LLM genera la risposta basandosi sui documenti

**Vantaggi:**
- Risposte basate su documenti specifici
- Riduce allucinazioni
- Permette di usare informazioni non presenti nel training

In [ ]:
# Verifica PDF
pdf_path = "../data/resources/acmecorp-employee-handbook.pdf"

if os.path.exists(pdf_path):
    print(f"✅ PDF trovato: {pdf_path}")
    print(f"   Dimensione: {os.path.getsize(pdf_path) / 1024:.1f} KB")
else:
    print(f"❌ PDF non trovato: {pdf_path}")

In [ ]:
# Installiamo le dipendenze per RAG
# uv pip install pypdf faiss-cpu langchain-community

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
# opzionale 
from langchain_huggingface import HuggingFaceEmbeddings

print("✅ Librerie RAG importate")

In [ ]:
# 1. Carica il PDF
loader = PyPDFLoader(pdf_path)
documents = loader.load()

print(f"✅ PDF caricato")
print(f"   Numero di pagine: {len(documents)}")
print(f"   Esempio contenuto prima pagina:\n")
print(documents[0].page_content[:300] + "...")

In [ ]:
# 2. Dividi il documento in chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Caratteri per chunk
    chunk_overlap=200,  # Sovrapposizione tra chunks
    length_function=len,
)

chunks = text_splitter.split_documents(documents)

print(f"✅ Documento diviso in chunks")
print(f"   Numero di chunks: {len(chunks)}")
print(f"   Esempio chunk:\n")
print(chunks[0].page_content)

In [ ]:
# 3. Crea embeddings e vector store
#embeddings = OpenAIEmbeddings()
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

print("🔄 Creazione vector store in corso (può richiedere alcuni secondi)...")
vectorstore = FAISS.from_documents(chunks, embeddings)

print("✅ Vector store FAISS creato!")
print(f"   Numero di vettori: {vectorstore.index.ntotal}")

In [ ]:
# 4. Test di ricerca semantica
query = "Qual è la politica delle ferie?"
relevant_docs = vectorstore.similarity_search(query, k=3)

print(f"🔍 Top 3 documenti rilevanti per: '{query}'\n")
for i, doc in enumerate(relevant_docs, 1):
    print(f"{i}. Pagina {doc.metadata.get('page', 'N/A')}:")
    print(f"   {doc.page_content[:200]}...\n")

In [ ]:
# 5. Creiamo un Retriever Tool
from langchain.tools import tool
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # Top 3 risultati
)

@tool
def employee_handbook_search(query: str) -> str:
    """Cerca informazioni nel manuale dei dipendenti di ACME Corp.
        
    Usa questo tool per rispondere a domande su:
    - Politiche aziendali
    - Benefit e ferie
    - Codice di condotta
    - Procedure HR

    Input: una domanda in linguaggio naturale"""
    try:
        docs = retriever.invoke(query)
        if not docs:
            return "Nessuna informazione trovata nel manuale."
        context = "\n\n".join([doc.page_content for doc in docs])
        return f"Informazioni trovate nel manuale:\n\n{context}"
    except Exception as e:
        return f"Errore durante la ricerca: {str(e)}"


print("✅ Retriever tool creato")

In [ ]:
# 6. Creiamo un RAG Agent con la nuova API
rag_agent = create_agent(
    model=model,
    tools=[employee_handbook_search],
    system_prompt="""Sei un assistente HR di ACME Corp specializzato nel manuale dei dipendenti.

Quando rispondi a domande:
1. Usa il tool di ricerca per trovare informazioni rilevanti nel manuale
2. Basa la tua risposta SOLO sulle informazioni trovate
3. Se le informazioni non sono nel manuale, dillo chiaramente
4. Cita sempre la fonte (pagina) delle informazioni

⚠️ IMPORTANTE: Non inventare informazioni. Se non sai qualcosa, ammettilo.

Rispondi sempre in italiano in modo professionale ma amichevole."""
)

print("✅ RAG Agent creato con la nuova API!")

In [ ]:
# Test RAG Agent
response = rag_agent.invoke({
    "messages": "Quanti giorni di ferie ho diritto?"
})

print("🤖 Risposta RAG Agent:\n")
print(response["messages"][-1].content)

In [ ]:
# Altra domanda
response = rag_agent.invoke({
    "messages": "Qual è la politica aziendale sul lavoro remoto?"
})

print("🤖 Risposta RAG Agent:\n")
print(response["messages"][-1].content)

In [ ]:
# Test con domanda fuori dal manuale
response = rag_agent.invoke({
    "messages": "Qual è lo stipendio medio in azienda?"
})

print("🤖 Risposta RAG Agent:\n")
print(response["messages"][-1].content)
print("\n💡 Nota: L'agente dovrebbe dire che questa informazione non è nel manuale")

## 💾 Salvataggio del Vector Store

Per evitare di ricreare gli embeddings ogni volta, salviamo il vector store:

In [ ]:
# Salva vector store su disco
vectorstore.save_local("../data/faiss_index")
print("✅ Vector store salvato in '../data/faiss_index'")

# Per ricaricare in futuro:
# vectorstore = FAISS.load_local(
#     "../data/faiss_index",
#     embeddings,
#     allow_dangerous_deserialization=True
# )

## 🎯 Esercizio Finale: Agente Multi-Tool

Combina tutti i tools in un unico agente super-potente!

**Challenge**: Crea un agente che può:
1. Cercare informazioni online (Tavily)
2. Interrogare il database Chinook
3. Rispondere su politiche aziendali (RAG)

**Esempio di interazione:**
- "Cerca online i migliori album del 2025, poi dimmi quali di questi artisti sono nel nostro database"
- "Qual è la nostra politica ferie e quanti clienti abbiamo nel database?"

In [ ]:
# Il tuo codice qui!

# Suggerimento: combina i tools
# all_tools = [search, retriever_tool] + sql_tools

# super_agent = create_agent(
#     model=model,
#     tools=all_tools,
#     system_prompt="..."
# )

## 📚 Riepilogo

In questa lezione hai imparato:

- ✅ **Ricerca Web**: Integrare Tavily per informazioni aggiornate
- ✅ **Summarization**: Usare `SummarizationMiddleware` ufficiale per conversazioni lunghe
- ✅ **Human-in-the-Loop**: Implementare con `interrupt()` di LangGraph
- ✅ **SQL Agent**: Interrogare database con linguaggio naturale
- ✅ **RAG**: Rispondere basandosi su documenti specifici con FAISS

### 🔑 Concetti Chiave

1. **Tavily > Google** per ricerche ottimizzate AI
2. **Nuova API `create_agent`** da `langchain.agents` (non più `langgraph.prebuilt`)
3. **SummarizationMiddleware** ufficiale con trigger multipli (messages, tokens, fraction)
4. **`interrupt()`** nativo per Human-in-the-Loop invece di decorator custom
5. **SQL Agents** democratizzano l'accesso ai dati
6. **RAG** riduce allucinazioni e permette knowledge base custom
7. **Vector stores** (FAISS) rendono la ricerca semantica efficiente

### 🆕 Novità API LangChain v1

**Cosa è Cambiato:**

| Vecchia API | Nuova API | Vantaggi |

|------------|-----------|----------|- 🔗 [Migration Guide](https://docs.langchain.com/oss/python/langchain/migration)

| `langgraph.prebuilt.create_react_agent` | `langchain.agents.create_agent` | Più semplice, unified interface |- 🔗 [LangGraph Interrupts](https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/)

| Middleware custom | `langchain.agents.middleware.*` | Standardizzati, testati, documentati |- 🔗 [SummarizationMiddleware](https://docs.langchain.com/oss/python/langchain/agents/middleware/summarization)

| Decorator custom | `interrupt()` da `langgraph.types` | Nativo, supporto checkpoint |- 🔗 [LangChain Agents Docs](https://docs.langchain.com/oss/python/langchain/agents)



**Parametri `create_agent`:**### 📝 Riferimenti

- `model`: Modello LLM (string o istanza)

- `tools`: Lista di tools- Gestione errori e retry avanzati

- `system_prompt`: Prompt di sistema (nuovo!)- Monitoring e observability

- `middleware`: Lista di middleware (nuovo!)- Deploy in produzione

- `checkpointer`: Per persistenza- Workflow orchestration con LangGraph

- `store`: Per storage cross-thread- Agenti multi-step complessi

- `interrupt_before/after`: Per human-in-the-loopNella prossima lezione:


### 🚀 Prossimi Passi